In [1]:
import torch
import torch.nn.functional as F
torch.manual_seed(1)

우선 임의의 3 × 5 행렬의 크기를 가진 텐서를 만듭니다.

In [4]:
z = torch.rand(3, 5, requires_grad=True)
print(z)

tensor([[0.4550, 0.5725, 0.4980, 0.9371, 0.6556],
        [0.3138, 0.1980, 0.4162, 0.2843, 0.3398],
        [0.5239, 0.7981, 0.7718, 0.0112, 0.8100]], requires_grad=True)


In [5]:
# 이제 이 텐서에 대해서 소프트맥스 함수를 적용합니다. 
# 단, 각 샘플에 대해서 소프트맥스 함수를 적용하여야 하므로 두번째 차원에 대해서 소프트맥스 함수를 적용한다는 의미에서 dim=1을 써줍니다.

hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.1664, 0.1871, 0.1737, 0.2695, 0.2033],
        [0.2002, 0.1783, 0.2218, 0.1944, 0.2054],
        [0.1809, 0.2380, 0.2318, 0.1084, 0.2409]], grad_fn=<SoftmaxBackward>)


이제 각 행의 원소들의 합은 1이 되는 텐서로 변환되었습니다. 소프트맥스 함수의 출력값은 결국 예측값입니다. 즉, 위 텐서는 3개의 샘플에 대해서 5개의 클래스 중 어떤 클래스가 정답인지를 예측한 결과입니다.

이제 각 샘플에 대해서 임의의 레이블을 만듭니다.

In [6]:
# row,high,size 를 인자로 받는 randint 임. 아래의 의미는 5미만의 정수를 1*3 텐서로 만들고 long 형으로 변환하라는 의미
y = torch.randint(5, (3,)).long()
print(y)

tensor([3, 1, 2])


이제 각 레이블에 대해서 원-핫 인코딩을 수행합니다.

In [7]:
# 모든 원소가 0의 값을 가진 3 × 5 텐서 생성
y_one_hot = torch.zeros_like(hypothesis) 

# unsqueeze 는 차원을 늘림. 즉 아래의 경우 dim=1 에 차원을 하나 늘려라는 의미임
# 그러면 y는 아래와 같이 변할 것임
# [[3],[1],[2]]

#scatter 는 dim,index,src를 인자를 받음
# 아래의 의미는 dim=1에 각각 해당하는 숫자의 열에 1을 집어넣으라는 의미 unsqueeze를 해야됨 그이유는 알아서 생각
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

각각 원소에 log(p) 를 곱한다. y_one_hot의 값이 1인 부분만 값이 나타나게 될것임

In [15]:
print("y_one_hot")
print(y_one_hot)
print()
print("hypothesis")
print(hypothesis)
print()
print("-log(hypothesis)")
print(-torch.log(hypothesis))
print()
print("y_one_hot * -log(hypothesis)")
print(y_one_hot*-torch.log(hypothesis))
print()
print("y_one_hot * -log(hypothesis).sum()")
print((y_one_hot * -torch.log(hypothesis)).sum(dim=1))
print()

print("cost")
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

y_one_hot
tensor([[0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

hypothesis
tensor([[0.1664, 0.1871, 0.1737, 0.2695, 0.2033],
        [0.2002, 0.1783, 0.2218, 0.1944, 0.2054],
        [0.1809, 0.2380, 0.2318, 0.1084, 0.2409]], grad_fn=<SoftmaxBackward>)

-log(hypothesis)
tensor([[1.7935, 1.6760, 1.7504, 1.3114, 1.5929],
        [1.6086, 1.7244, 1.5062, 1.6381, 1.5826],
        [1.7096, 1.4354, 1.4617, 2.2223, 1.4236]], grad_fn=<NegBackward>)

y_one_hot * -log(hypothesis)
tensor([[0.0000, 0.0000, 0.0000, 1.3114, 0.0000],
        [0.0000, 1.7244, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.4617, 0.0000, 0.0000]], grad_fn=<MulBackward0>)

y_one_hot * -log(hypothesis).sum()
tensor([1.3114, 1.7244, 1.4617], grad_fn=<SumBackward1>)

cost
tensor(1.4992, grad_fn=<MeanBackward0>)


In [16]:
# 네번째 수식
F.cross_entropy(z, y)

tensor(1.4992, grad_fn=<NllLossBackward>)

# F.cross_entropy는 비용 함수에 소프트맥스 함수까지 포함하고 있음을 기억하고 있어야 구현 시 혼동하지 않습니다.